### 교차 검증

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression


iris = load_iris()
iris_data = iris.data
target = iris.target

lr_clf = LogisticRegression(max_iter=1000)
score = cross_val_score(lr_clf, iris_data, target)
print("교차 검증 점수:",score)


교차 검증 점수: [0.96666667 1.         0.93333333 0.96666667 1.        ]


#### 교차 검증에 fold 수 바꾸기

In [3]:
score = cross_val_score(lr_clf, iris_data, target, cv=5) # fold 5개
print("교차 검증 점수: ", score)
print("평균 교차 검증 점수: {0:.4f}".format(round(np.mean(score), 4)))

교차 검증 점수:  [0.96666667 1.         0.93333333 0.96666667 1.        ]
평균 교차 검증 점수: 0.9733


#### cross_validate

In [4]:
# cross_val_score 함수와 인터페이스가 비슷하지만 훈련과 테스트에 걸린 시간을 담은 딕셔너리 반환
from sklearn.model_selection import cross_validate

res = cross_validate(lr_clf, iris_data, target, cv=5, return_train_score=True)
res

{'fit_time': array([0.01100087, 0.01400375, 0.00900292, 0.00900102, 0.00900221]),
 'score_time': array([0.        , 0.00100017, 0.        , 0.        , 0.        ]),
 'test_score': array([0.96666667, 1.        , 0.93333333, 0.96666667, 1.        ]),
 'train_score': array([0.96666667, 0.96666667, 0.98333333, 0.98333333, 0.975     ])}

In [5]:
# dataframe으로 만들고 각 컬럼별 평균 반환하기
res_df = pd.DataFrame(res)
display(res_df)
print('평균시간과 점수\n', res_df.mean())

,fit_time,score_time,test_score,train_score
0,0.011001,0.000,0.966667,0.966667
1,0.014004,0.001,1.000000,0.966667
2,0.009003,0.000,0.933333,0.983333
3,0.009001,0.000,0.966667,0.983333
4,0.009002,0.000,1.000000,0.975000


평균시간과 점수
 fit_time       0.010402
score_time     0.000200
test_score     0.973333
train_score    0.975000
dtype: float64


#### 계층별 k-fold 교차 방법 (Stratified K 폴드)

In [7]:
from sklearn.datasets import load_iris

iris = load_iris()
print('Iris 레이블:\n', iris.target)


Iris 레이블:
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [9]:
# 단순한 KFold 하기
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

# DataFrame으로 만들기
iris_df = pd.DataFrame(iris_data, columns=iris.feature_names)
iris_df['target'] = target

# Kfold 생성
kf = KFold(n_splits=3)
n_iter = 0

for train_idx, val_idx in kf.split(iris_df):
    n_iter += 1
    y_train, y_val = iris_df['target'].iloc[train_idx], iris_df['target'].iloc[val_idx]
    print('## 교차 검증 KFold {0} ##'.format(n_iter))
    print("## 학습 데이터 ##\n {0}".format(y_train.value_counts()))
    print("## 검증 데이터 ##\n {0}".format(y_val.value_counts()))
    

## 교차 검증 KFold 1 ##
## 학습 데이터 ##
 1    50
2    50
Name: target, dtype: int64
## 검증 데이터 ##
 0    50
Name: target, dtype: int64
## 교차 검증 KFold 2 ##
## 학습 데이터 ##
 0    50
2    50
Name: target, dtype: int64
## 검증 데이터 ##
 1    50
Name: target, dtype: int64
## 교차 검증 KFold 3 ##
## 학습 데이터 ##
 0    50
1    50
Name: target, dtype: int64
## 검증 데이터 ##
 2    50
Name: target, dtype: int64


In [10]:
# kfold로 교차 검증 점수 얻기
kf = KFold(n_splits=3)

print('교차 검증 점수:\n', cross_val_score(lr_clf, iris_data, target, cv=kf))
# 각 폴드는 iris 데이터 셋 클래스 중 하나에 대응하므로 아무것도 학습할 수 없다. 

교차 검증 점수:
 [0. 0. 0.]


In [11]:
# Kfold 데이터 분할 전 섞어주기
kf = KFold(n_splits=3, shuffle=True, random_state=0)
print('교차 검증 점수:\n', cross_val_score(lr_clf, iris_data, target, cv=kf))

교차 검증 점수:
 [0.98 0.96 0.96]


#### 클래스의 같은 비율로 폴드 나누기

In [12]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n_iter = 0

for train_idx, val_idx in skf.split(iris_df, iris_df['target']):
    n_iter += 1
    y_train, y_val = iris_df['target'].iloc[train_idx], iris_df['target'].iloc[val_idx]
    print('## 교차 검증 KFold {0} ##'.format(n_iter))
    print("## 학습 데이터 ##\n {0}".format(y_train.value_counts()))
    print("## 검증 데이터 ##\n {0}".format(y_val.value_counts()))
# 클래스 비율에 맞춰서 폴드 나눠짐

## 교차 검증 KFold 1 ##
## 학습 데이터 ##
 2    34
0    33
1    33
Name: target, dtype: int64
## 검증 데이터 ##
 0    17
1    17
2    16
Name: target, dtype: int64
## 교차 검증 KFold 2 ##
## 학습 데이터 ##
 1    34
0    33
2    33
Name: target, dtype: int64
## 검증 데이터 ##
 0    17
2    17
1    16
Name: target, dtype: int64
## 교차 검증 KFold 3 ##
## 학습 데이터 ##
 0    34
1    33
2    33
Name: target, dtype: int64
## 검증 데이터 ##
 1    17
2    17
0    16
Name: target, dtype: int64


#### LOOCV(Leave-one-out Cross-validation)

In [14]:
from sklearn.model_selection import LeaveOneOut
# loo 선언
loo = LeaveOneOut()
scores = cross_val_score(lr_clf, iris_data, target, cv=loo)
print('교차 검증 분할 횟수:', len(scores))
print('교차 검증 분할 scores:', scores)
print("평균 정확도 : {0:.2f}".format(scores.mean()))

교차 검증 분할 횟수: 150
교차 검증 분할 scores: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.]
평균 정확도 : 0.97


#### shuffle-split cross-validation

In [15]:
from sklearn.model_selection import ShuffleSplit
# 데이터셋의 50%를 train 세트, 50%를 test세트로 10번 분할
shuffle_split = ShuffleSplit(test_size=.5, train_size=.5, n_splits=10)
scores = cross_val_score(lr_clf, iris_data, target, cv=shuffle_split)
print("교차 검증 점수:\n", scores)

교차 검증 점수:
 [0.98666667 0.97333333 0.94666667 0.93333333 0.92       0.97333333
 0.94666667 0.92       0.97333333 0.96      ]


#### GroupKFold

In [16]:
from sklearn.model_selection import GroupKFold
from sklearn.datasets import make_blobs
gkf = GroupKFold(n_splits=3)

# 인위적인 데이터 생성
X, y = make_blobs(n_samples=12, random_state=0)
# 처음 세 개의 샘플은 같은 그룹에 속함
groups= [0,0,0,1,1,1,1,2,2,3,3,3]
scores = cross_val_score(lr_clf, X, y, groups=groups, cv=gkf)

print("교차 검증 점수:\n", scores)

교차 검증 점수:
 [0.75       0.6        0.66666667]


#### 반복 교차 검증 - RepeatedStratifiedKFold

In [17]:
from sklearn.model_selection import RepeatedStratifiedKFold

rskf = RepeatedStratifiedKFold(random_state=42)
scores = cross_val_score(lr_clf, iris_data, target, cv=rskf)

print("교차 검증 점수:\n", scores)
print("교차 검증 평균 점수: {:.3f}".format(scores.mean()))

교차 검증 점수:
 [1.         0.96666667 0.93333333 1.         0.93333333 0.96666667
 0.96666667 0.93333333 1.         0.96666667 0.93333333 1.
 1.         0.96666667 0.96666667 0.9        1.         1.
 0.93333333 0.96666667 0.93333333 0.96666667 0.96666667 1.
 0.96666667 1.         0.96666667 0.96666667 0.9        1.
 0.96666667 0.96666667 0.96666667 0.96666667 0.93333333 0.96666667
 0.96666667 1.         1.         0.9        0.96666667 1.
 0.9        0.96666667 0.96666667 0.9        0.96666667 0.96666667
 1.         0.96666667]
교차 검증 평균 점수: 0.965
